In [ ]:
#[[1922,50,16]]  Algorithm 4 and 5
#This is the final version of the code, as some operations during my modification process did not affect the results, I did not delete them.
from sympy import *
from sympy import Matrix, zeros
from sympy import eye
import numpy as np
import matplotlib.pyplot as plt
import math
from sympy import symbols, Poly
import seaborn as sns
from sympy.polys.galoistools import gf_gcd, gf_div, gf_gcdex
from sympy.polys.domains import ZZ
from numpy.random import SeedSequence, default_rng
import multiprocessing

#matrix in GF(2)
def mod2(matrix):
    """Reduce a matrix to modulo 2"""
    return np.mod(matrix, 2)
# standard form (I_k P)
def gf2_rref(matrix):
    """Compute the reduced row echelon form (RREF) of a matrix in GF(2)"""
    matrix = mod2(matrix)
    rows, cols = matrix.shape
    row, col = 0, 0
    
    while row < rows and col < cols:
        if matrix[row, col] == 0:
            for r in range(row + 1, rows):
                if matrix[r, col] == 1:
                    matrix[[row, r]] = matrix[[r, row]]
                    break
        if matrix[row, col] == 1:
            for r in range(rows):
                if r != row and matrix[r, col] == 1:
                    matrix[r] = mod2(matrix[r] + matrix[row])
            row += 1
        col += 1

    return matrix
# the kernel of matrix  (P^T I_{n-k})
def kernel_gf2(matrix):
    """Compute the kernel of a matrix in GF(2)"""
    matrix_rref = gf2_rref(matrix)
    rows, cols = matrix_rref.shape
    pivot_cols = []

    for r in range(rows):
        for c in range(cols):
            if matrix_rref[r, c] == 1:
                pivot_cols.append(c)
                break

    free_vars = [c for c in range(cols) if c not in pivot_cols]
    kernel_basis = []
    #print(free_vars)
    for free_var in free_vars:
        basis_vector = np.zeros(cols, dtype=int)
        basis_vector[free_var] = 1
        for pivot_col in pivot_cols:
            row = pivot_cols.index(pivot_col)
            if matrix_rref[row, free_var] == 1:
                basis_vector[pivot_col] = 1
        kernel_basis.append(basis_vector)

    return np.array(kernel_basis).T
# the circulant matrix A = polynomial_to_circulant(poly, l).T  can be uniquely represented by the polynomial "poly"
def polynomial_to_circulant(poly, l):
   
    x = symbols('x')

   
    if not isinstance(poly, Poly):
        poly = Poly(poly, x,domain=ZZ)

    
    coeffs = poly.all_coeffs()

    # 系数需要从低次幂到高次幂排列
    coeffs.reverse()

    # Create a l \times l matrix
    circulant_matrix = np.zeros((l, l), dtype=int)

    # Fill the elements to get the circulant of  the "poly"
    for i in range(l):
        a1 = np.pad( coeffs, (0, l - len( coeffs)), 'constant')
        circulant_matrix[i] = row = np.roll(a1, i)

    return circulant_matrix

#from (I_k P) to (0 I_{n-k})
def make_trailing_columns_zero(A):
    """
    Modify the matrix A such that all trailing columns (after the first m columns) are zero.

    Args:
    - A: numpy array, the input matrix

    Returns:
    - A_modified: numpy array, the modified matrix with trailing columns zeroed out
    """
    A1 = np.matrix(np.array(A.tolist()))
    m, n = A1.shape
    A1 = gf2_rref(A1)
    A1_modified = np.copy(A1)  # Create a copy to avoid modifying the original matrix
    last_row = A1_modified[-1]
    indices = np.nonzero(last_row)[0]
    l = indices[0] 
    A1_modified[:, l+1:] = 0  
    return A1_modified
#  check matrices of two binary linear codes H_1 and H_2 which  generate a hypergraph product code
x = symbols('x')
n =31
k = 5
# 求出g（X）
h_X = 1 + x**2 + x**5 
H_X =x**k*h_X.subs(x, 1/x)
R_X = 1 + x**31
Hn = polynomial_to_circulant(h_X, n) # H1
# H_1
Hn_T = polynomial_to_circulant(H_X, n) #H2  noticing H1.T = H2
rowspaceH4 = Hn_T[:n-k]

b = Poly(h_X, x, domain=ZZ).all_coeffs()
r = Poly(R_X, x, domain=ZZ).all_coeffs()
#a.reverse()
b.reverse()
r.reverse()
G_X,_ = gf_div(r,b , 2, ZZ)
#g_X*h_X = R_X
g_X = sum(coef * x**i for i, coef in enumerate(G_X))
G_X =x**(n-k)*g_X.subs(x, 1/x)

G = polynomial_to_circulant(g_X, n)
G = G.T # the kernel of H1


G_T = polynomial_to_circulant(G_X, n)
G_T = G_T.T
G = G[:k]
G_T = G_T[:k] # the kernel of H2


rowspaceH3 = Hn[:n-k]
# we can obtain  the generate matrix of a cyclic code in the form (I P) or (P_1 I)
# M is in the form (0 I), which is F_2^n/rs(H1)
if __name__ == "__main__":
    
    A = rowspaceH3
    # Make trailing columns zero
    A1_modified = make_trailing_columns_zero(A)
    A1_modified2 = Matrix( A1_modified.tolist())
    #print("Original matrix A:")
    #print(A)
   # print("矩阵后m行元素全为0:")
    #print(A1_modified)
   
M = Matrix()


for i in range(n):  # 遍历单位矩阵的每一列
    e_i = eye(n)[i, :]  # 单位矩阵的第 i 列
    
    is_linearly_dependent = any(e_i==(A1_modified2.row(j)) for j in range(A1_modified.shape[0]))
    
    if not is_linearly_dependent:
        
        M = M.row_insert(M.rows, e_i)
print(M)
M1 = np.matrix(np.array(M.tolist()))
# N is in the form (0 I), which is F_2^n/rs(H2)
if __name__ == "__main__":
    A11_modified = make_trailing_columns_zero(rowspaceH4)
    A11_modified2 = Matrix( A11_modified.tolist())
N = Matrix()
for i in range(n):  
    e_i = eye(n)[i, :]  
    
    is_linearly_dependent = any(e_i==(A11_modified2.row(j)) for j in range(A11_modified.shape[0]))
    
    if not is_linearly_dependent:
        N = N.row_insert(N.rows, e_i)
print(N)
N1 = np.matrix(np.array(N.tolist()))
#  obtain  the generate matrix G and G_T of the cyclic codes in the form (P_1 I)
original_list = list(range(n))
reversed_list = original_list[::-1]
Ker_H = G[:, reversed_list]
Ker_H = Matrix(Ker_H.tolist())
Ker_H = gf2_rref(Ker_H)
Ker_H1T = G_T[:, reversed_list]
Ker_H1T = Matrix(Ker_H1T.tolist())
Ker_H1T = gf2_rref(Ker_H1T)
original_listrow = list(range(5))
reversed_listrow = original_listrow[::-1]
Ker_H = Ker_H[reversed_listrow, :]

Ker_H1T =Ker_H1T[reversed_listrow, :]
Ker_H = Ker_H[:, reversed_list]
Ker_H1T = Ker_H1T[:, reversed_list]
print(Ker_H )
print(Ker_H1T )
# M is the right reverse of  Ker_H (G1)
# N is the right reverse of  Ker_H (G2)
Ker_H = np.matrix(np.array(Ker_H.tolist()))
Ker_H1T = np.matrix(np.array(Ker_H1T.tolist()))
# V \times V
LogicalX = np.kron(Ker_H, M1)
LogicalZ = np.kron(M1,Ker_H )
#C\times C
LogicalX1 = np.kron(N1,Ker_H1T )
LogicalZ1 = np.kron(Ker_H1T,N1 )  
O_X = np.zeros((25,31*31))
def swap_columns(matrix, col1, col2):
    matrix[:, [col1, col2]] = matrix[:, [col2, col1]]
    return matrix
def swap_rows(matrix, row1, row2):
    matrix[[row1,row2],:] = matrix[[row2,row1],:]
    return matrix
#by permutations of rows and columns to get the  Standard from
def create_identity_at_top(matrix):
    """Ensure the first k rows and k columns form an identity matrix using column swaps"""
    rows, cols = matrix.shape
    P = np.eye(cols)
    
    for i in range(rows):
        if matrix[i, i] != 1:
            # Find a column with a 1 in the i-th row
            for j in range(i + 1, cols):
                if matrix[i, j] == 1:
                    swap_columns(matrix, i, j)
                    swap_rows(P,i, j)
                    break
    return matrix,P
max_nonzero_per_row = 4 # L = max_nonzero_per_row + !
def generate_generator_matrix(k,seed):
    # Generate a random k x n binary domain matrix, requiring the matrix to be full rank
    rng = default_rng(seed)
    n = 25*2 
    P = np.zeros((k, n-k), dtype=int)
    for i in range(k):
        non_zero_count = rng.integers(0, max_nonzero_per_row + 1)  
        non_zero_indices = rng.choice(n-k, non_zero_count, replace=False)  
        P[i, non_zero_indices] = 1  # 在GF(2)中，非零元值只能是1    
    I_n = np.eye(n)
    P_T=P.T
   
    permutation = rng.permutation(n)
    
    M = I_n[permutation,:]
    
    I = np.eye(k)
    I_T = np.eye(n-k)
    O_nk = np.zeros((k, n-k))
    O_k = np.zeros((n-k, k))
    V1_X =  np.hstack((I, P))
    
    V1_Z =  np.hstack((O_k, I_T))
    V_X = V1_X @ M
    V_Z = V1_Z @ M
    U1_X = np.hstack((P_T, I_T))
    U1_Z = np.hstack((I, O_nk))
    
    U_X = U1_X @ M
    U_Z = U1_Z @ M
    return V_X , V_Z , U_X , U_Z
LogicalX0 = np.hstack((LogicalX , O_X ))
LogicalZ0 = np.hstack((LogicalZ , O_X ))
LogicalX10 = np.hstack((O_X , LogicalX1 ))
LogicalZ10 = np.hstack((O_X , LogicalZ1 ))
# logical operators J_X,J_Z
LogicalX_q =  np.vstack((LogicalX0 , LogicalX10 ))
LogicalZ_q =  np.vstack((LogicalZ0 , LogicalZ10 ))
results_0 = []
results1_0 = []
results1_1 = []
def thecost(v,seed):
   V_X , V_Z , U_X , U_Z  = generate_generator_matrix(v,seed)
   LogicalX_qs0 =np.dot(V_X , LogicalX_q)%2 #J_{Z,A}
   LogicalZ_qs0  = np.dot(U_X , LogicalZ_q)%2 
   LogicalX_qs1  = np.dot(V_Z , LogicalX_q)%2 #J_{Z,C } "supplemental " Algorithm 4 
   LogicalX_qs=np.array(LogicalX_qs0.tolist())
   LogicalZ_qs=np.array(LogicalZ_qs0.tolist())
   LogicalX_qs1=np.array(LogicalX_qs1.tolist())
   #non_zero_columns = 0
   LogicalX_qs=np.array(LogicalX_qs0.tolist())
   LogicalZ_qs=np.array(LogicalZ_qs0.tolist())
   LogicalX_qs1=np.array(LogicalX_qs1.tolist())
#  get V_XJ_X S^T 
   non_zero_cols = [col_idx for col_idx in range(LogicalX_qs0.shape[1]) if not all(row[col_idx] == 0 for row in LogicalX_qs)]
   gg = len(non_zero_cols)
   
   delelogicalX = LogicalX_qs[:,non_zero_cols ] #V_XJ_X S^T 
   delelogicalZ = LogicalZ_qs[:,non_zero_cols ] #U_XJ_Z S^T 
   delelogicalX1 = np.array(delelogicalX.tolist())
   delelogicalX2 = delelogicalX1[~np.all(delelogicalX1 == 0, axis=1)]#消除全为零元的行，这步操作可以没有，因为无全零行
   f = delelogicalZ.shape[1]
   n = 31
   I = np.eye(n)
   H_Z1 = np.kron(Hn, I)
   H_Z2 = np.kron(I, Hn_T)
   H_Z = np.hstack((H_Z1,H_Z2))#get the parity check matrix H_Z
   filtered_matrix = H_Z[:,non_zero_cols] # H_ZS^T
   filtered_matrix1 = np.array(filtered_matrix.tolist())
   filtered_matrix2 = filtered_matrix1[~np.all(filtered_matrix1 == 0, axis=1)]#get H_N  "supplemental " equation(24)
   saved_rows = []
   
   saved_rows1 = []
   K = kernel_gf2(filtered_matrix2).T #the kernel of H_N
# to get the G2: G2(J_{Z,C}S^T)^T = U and U is full rank 
   for row in K:
    
        row_matrix = row.reshape(1, -1)
   
        result_vector = np.dot(row_matrix,delelogicalZ.T) % 2
    
        if np.any(result_vector != 0):        
           saved_rows2 = [r.flatten() for r in saved_rows1]  # 转换为列表
           F = np.array(saved_rows)
           m4 = F.shape[0]
           saved_rows2.append(result_vector.flatten())
           saved_matrix2 = np.array(saved_rows2) 
           saved_matrix2 = gf2_rref(saved_matrix2)
           all_nonzero_rows = saved_matrix2[np.any(saved_matrix2 != 0, axis=1)]
           m3 = all_nonzero_rows.shape[0]
           if m3 > m4:
             saved_rows.append(row)
             saved_rows1.append(result_vector.flatten())

#Algorithm 4 step 6-9
   delelogicalZ1 = np.array(saved_rows)
   saved_matrix1 = np.array(saved_rows1)
   G_12 = delelogicalX2
   if delelogicalZ1.shape[0] != 0:
        G_12 = np.vstack(( delelogicalX2,delelogicalZ1))
   
    
   m_N = G_12.shape[0]
   
   G_12 = gf2_rref(G_12)
   G3 ,P1 = create_identity_at_top(G_12)
   
   O_N = np.zeros((m_N,f-m_N))
   M = np.hstack((eye(m_N),O_N))
   barG1 = np.dot(delelogicalX2,P1.T)
   barG1 = np.dot(barG1,M.T)%2
   barD = kernel_gf2( barG1 ).T
   m2 = 0
   D = []
# noticing step 10 and equtation (101) and (105) :Sparsing matrix D is the same as sparsifying matrix barD, so to simplify that  we make D =barD. 
   if  barD.shape[0] != 0 :
       D = barD
       D = np.array(D.tolist())
       m2 = barD.shape[0]
   m = filtered_matrix2.shape[0]
   
   m1 = filtered_matrix2.shape[1]
   Y1 = m1*15 + m2*16 + m*16 #16 is the code distance
   if m2 != 0 :
     h = D.shape[1]
     count_col = [0] * h
   
    #the qubits cost of Sparsing matrix D and weight(H_G) < = weight(H_X)+2
     for row in D:
        count = 0
        for element in row:
            if element != 0:
                count += 1
        if count > 6:
            log2_count = math.log(count, 5)
            g = math.ceil(log2_count)
            m1 = m1 + 5*(1-5**(g-1)) / (1-5) 
            m2 = m2 + 5*(1-5**(g-1)) / (1-5)
     for j in range(D.shape[1]):
        for i in range(D.shape[0]):  
            if D[i][j] != 0:  
                count_col[j] += 1 
     for j in range(D.shape[1]):
        if count_col[j] > 3:
            log3_count_col = math.log(count_col[j], 2)
            f = math.ceil(log3_count_col)
            m2 = m2  + 2*(1-2**(f-1)) / (1-2)
            m1 = m1  + 2*(1-2**(f-1)) / (1-2)
   Y = m1*15+ m2*16 + m*16
   X = 1#no meaning
   result = (Y1,Y,gg,0) 
   
   return result
def parallel_computation(data):
    seeds = SeedSequence().spawn(len(data))
    args = [(v,seed) for v,seed in zip(data,seeds)]
    with multiprocessing.Pool(processes=50) as pool:
        result =  pool.starmap(thecost, args)
    return result
if __name__ == "__main__":
    dates = [20]*100 # 20 is the number of logical operators,we can make q = 1,5,10,20,30,40.
    results = parallel_computation(dates)
    
g1 = [res[2] for res in results]
g2 = [res[3] for res in results]
Y_values = [result[1] for result in results]
Y1_values = [result[0] for result in results]

print(results)

sns.displot(Y_values, bins=20, kde=True, color='green')
plt.title('Histogram with Density Plot')
plt.xlabel('cost')
plt.ylabel('Frequency')
plt.show()
sns.displot(Y1_values, bins=20, kde=True, color='blue')
plt.title('Histogram with Density Plot2')
plt.xlabel('cost')
plt.ylabel('Frequency')
plt.show()

resultsall=[]
resultsall1=[]

median_Y = np.percentile(Y_values, 50)
resultsall.append((20,median_Y ))
median_Y1 = np.percentile(Y1_values, 50)
resultsall1.append((20,median_Y1 ))


print(resultsall)
print(resultsall1)

X_values = [result[0] for result in resultsall]
Y1_values = [result[1] for result in resultsall1]
Y_values = [result[1] for result in resultsall]
plt.figure()
plt.plot(X_values, Y_values, marker='o',label='D-no sparse matrix')
plt.plot(X_values, Y1_values, marker='*',label='D-sparse matrix')
plt.title('Algorithm 4 and 5')
plt.xlabel('the number of logical operators')
plt.ylabel('qubit cost ')
plt.legend()  
plt.show().
